In [1]:
import base as b
import pandas as pd
import datetime as dt
from typing import Dict
import talib as ta
import utils as ut
from constants import *
from scipy.stats import linregress
from logger_settings import logger


pd.options.plotting.backend = 'holoviews'

settings = {
    "vwch_thresh": 3,
    "vrf": 1, # volume_retrace_factor
    "vsf": 3, # volume_stagnate_factor
    "quantity": 50 * 1, # 1 lot
    "stagnation_cooldown": 5, # cooldown period after stagnation to resume, in ticks
    "svs_mn_thresh": 5000, # stagnation_volume_std_max_threshold
    "svs_mx_thresh": 10000, # stagnation_volume_std_min_threshold
}

instrument = b.Instrument(name="NIFTY 22650 CALL 7 Mar 2024")

SYMBOL = "NIFTY"
expiry = dt.datetime.strptime("2024-03-21", "%Y-%m-%d").date()
otype = "CE"
strike = 22000
date = dt.datetime.strptime("2024-03-15", "%Y-%m-%d").date()

tdf = ut.get_ticks(symbol=SYMBOL, expiry=expiry, strike=strike, otype=otype, date=date)

In [2]:
REASON_REVERSE_SIGNIFICANCE = "reverse significance found"
REASON_STAGNATING = "trend stagnating"
REASON_CONFIRM = "confirmed"

class Phase(b.BasePhase):
    def __init__(self, initiated_at, strategy):
        super().__init__(initiated_at, strategy)
        self.volume_confirmed = False
        self.parent_volume_confirmed = False
        self.last_slope = None
        self.stagnated_at = None
        self.stagnate_reason = None
        logger.info(f"created new phase{self.id} at: {initiated_at}")
        if initiated_at < strategy.ticks.iloc[-1].name:
            self.rerun()

    def __repr__(self):
        return (f"==Phase {self.id}:{self.direction}, {self.status} I:{self.initiated_at}, R: {self.resumed_at}, C:{self.confirmed_at} R:{self.retraced_at}, "
               f"T:{self.term_at}, RJ:{self.rejected_at}, cr:{self.created_at} tr: {self.term_reason}, rr:{self.retrace_reason}, "
               f"end:{self.ended_at}, vc: {self.volume_confirmed}, pc: {self.parent_volume_confirmed}, st: {self.stagnated_at}==")

    def rerun(self):
        self.next()

    def has_breached_start(self):
        up_breach = self.direction == UP and (self.strategy.ticks.iloc[-1].last_price <= self.strategy.ticks.loc[self.initiated_at].last_price).all()
        if up_breach:
            return up_breach
        down_breach = self.direction == DOWN and (self.strategy.ticks.iloc[-1].last_price >= self.strategy.ticks.loc[self.initiated_at].last_price).all()
        return down_breach

    def get_new_phase_start(self):
        """
        Return new phase start when current phase is breached
        """
        if self.direction == UP:
            return self.strategy.ticks.loc[self.strategy.ticks.index > self.resumed_at].last_price.idxmax()
        else:
            return self.strategy.ticks.loc[self.strategy.ticks.index > self.resumed_at].last_price.idxmin()

    def is_tick_significant(self):
        """
        If a tick has a very high vwch, that could mean that its a very significant tick.
        If opposite direction tick has a very high vwch, its a good sign of reversal.
        Returns -1 if its opposite direction significant
        Returns 1 if same direction significant
        Returns 0 if not significant
        """
        thresh = self.strategy.vwch_thresh
        is_volume_significant = self.pticks.iloc[-1].volume >= (thresh * self.pticks.volume.mean())
        result = 1
        if not is_volume_significant:
            result = 0
        elif self.direction == UP and self.pticks.iloc[-1].ch <= 0:
            result = -1
        elif self.direction == DOWN and self.pticks.iloc[-1].ch >= 0:
            result = -1
        logger.info(f"is_significant: {result}, at:{self.pticks.iloc[-1].name}, ptick start: {self.pticks.iloc[0].name} mean: {self.pticks.volume.mean()}, v: {self.pticks.iloc[-1].volume}, ch: {self.pticks.iloc[-1].ch}, d: {self.direction}")
        return result

    def terminate(self, reason):
        self.status = self.STATUS_TERM
        self.term_at = self.strategy.ticks.iloc[-1].name
        self.term_reason = reason
        if self.retraced_at is None:
            self.retraced_at = self.strategy.ticks.iloc[-1].name
            self.retrace_reason = "on phase termination"
        self.strategy.on_termination(self, reason)

    def retracing(self, reason):
        self.status = self.STATUS_RETRACING
        # self.volume_confirmed = False
        self.retraced_at = self.strategy.ticks.iloc[-1].name
        self.retrace_reason = reason
        self.strategy.on_retracel(self, reason)

    def stagnating(self, reason):
        if self.status == self.STATUS_STAGNATING:
            return
        self.status = self.STATUS_STAGNATING
        self.stagnated_at = self.strategy.ticks.iloc[-1].name
        self.stagnate_reason = reason
        self.strategy.on_stagnation(self, reason)

    def confirm(self):
        if self.status == self.STATUS_RETRACING:
            self.resumed_at = self.pticks.iloc[-1].name
        self.status = self.STATUS_CONFIRMED
        self.strategy.on_confirmed(self, REASON_CONFIRM)

    def calculate_pdata(self):
        self.pticks["ch"] = self.pticks.last_price.diff()
        self.pticks["chv"] = self.pticks.volume  # Volume weighted change
        self.pticks.loc[self.pticks.ch < 0, "chv"] = -self.pticks.volume
        # self.pticks["cum_vwch"] = self.pticks.vwch.cumsum()
        # self.pticks["std_vwch"] = self.pticks.cum_vwch.rolling(window=5).std()  # Volume weighted change
        # self.last_slope = self.get_slope(self.ended_at)

    def get_slope(self, eidx):
        if self.direction == UP:
            last_idx = self.pticks.loc[:eidx-1].cum_vwch.idxmax()
        else:
            last_idx = self.pticks.loc[:eidx-1].cum_vwch.idxmin()
        if pd.isna(last_idx) or eidx - last_idx < 4:
            return
        df = self.pticks.loc[last_idx: eidx]
        factor = 1.6
        if self.status == self.STATUS_STAGNATING:
            multiplier = self.pticks.loc[:self.stagnated_at].vwch.mean() * factor
        else:
            multiplier = self.pticks.vwch.mean() * factor
        multiplier = abs(multiplier * (7 / (eidx - last_idx)))
        try:
            lr = linregress(df.id * multiplier, df.cum_vwch)
        except ValueError:
            raise Exception(f"asd: {last_idx}, {eidx}")
        logger.info(f"slope: {last_idx} to {eidx}: {lr.slope}, init: {self.pticks.iloc[0].name}, m: {multiplier}, {self}")
        return lr.slope

    def is_retracing(self):
        if self.direction == UP:
            is_re = self.pticks.iloc[-1].chv < - self.pticks.volume.max() * self.strategy.vrf
        else:
            is_re = self.pticks.iloc[-1].chv > self.pticks.volume.max() * self.strategy.vrf
        logger.info(f"is_retracing: {self.pticks.iloc[-1].name}, is_re: {is_re}, chv: {self.pticks.iloc[-1].chv}, vmax: {self.pticks.volume.max()} vrf: {self.strategy.vrf}")
        return is_re

    def is_stagnating(self):
        if self.direction == UP:
            is_st = self.pticks.iloc[-1].chv < - self.pticks.volume.mean() * self.strategy.vsf
        else:
            is_st = self.pticks.iloc[-1].chv > self.pticks.volume.mean() * self.strategy.vsf
        logger.info(f"is_stagnating: {self.pticks.iloc[-1].name}, is_st: {is_st}, chv: {self.pticks.iloc[-1].chv}, vmean: {self.pticks.volume.mean()} vsf: {self.strategy.vsf}")
        return is_st
        # if self.status == Phase.STATUS_STAGNATING:
        #     logger.info(f"stagnating1? {self.pticks.iloc[-1].name}, {self.pticks.iloc[-1].std_vwch}: {self.pticks.iloc[-1].std_vwch < self.strategy.svs_mx_thresh}")
        #     return self.pticks.iloc[-1].std_vwch < self.strategy.svs_mx_thresh and (self.ended_at - self.stagnated_at >= self.strategy.stagnation_cooldown)
        # logger.info(f"stagnating2? {self.pticks.iloc[-1].name}, {self.pticks.iloc[-1].std_vwch}: {self.pticks.iloc[-1].std_vwch <= self.strategy.svs_mn_thresh}")
        # return self.pticks.iloc[-1].std_vwch <= self.strategy.svs_mn_thresh

    def has_resumed(self):
        if self.direction == UP:
            is_re = self.pticks.last_price.iloc[-1] >= self.pticks.last_price.max()
        else:
            is_re = self.pticks.last_price.iloc[-1] <= self.pticks.last_price.min()
        logger.info(f"has_resumed: {self.pticks.iloc[-1].name}, is_re: {is_re}, lp: {self.pticks.last_price.iloc[-1]}, pmax: {self.pticks.last_price.max()} pmin: {self.pticks.last_price.min()}")
        return is_re

    def resume(self):
        self.status = self.STATUS_CONFIRMED
        self.resumed_at = self.pticks.iloc[-1].name
        self.strategy.on_resume()

    def next(self):
        super().next()
        self.calculate_pdata()
        if self.status == self.STATUS_TERM:
            raise Exception(f"phase {self} already terminated")
        if self.status == self.STATUS_INITIATED:
            if self.pticks.iloc[-1].ch > 0:
                self.direction = UP
            else:
                self.direction = DOWN
            self.confirm()
            return
        if self.has_breached_start():
            self.terminate("breached start")
            return
        if self.status == self.STATUS_RETRACING:
            return
        significance = self.is_tick_significant()
        is_retr = self.is_retracing()
        if significance == -1 or is_retr:
            self.retracing(REASON_REVERSE_SIGNIFICANCE)
            return
        elif significance == 1:
            pass
            # self.volume_confirmed = True
        if self.status == self.STATUS_STAGNATING:
            raise Exception("this shouldn't be reached")
            if self.has_resumed():
                self.resume()
            return
        is_stagnating = self.is_stagnating()
        if is_stagnating:
            self.stagnating(REASON_STAGNATING)
            return
        self.confirm()

class PhaseStrategy(b.BasePhaseStrategy):
    def __init__(self, instrument: "Instrument", settings: Dict):
        super().__init__(instrument, settings)
        self.om = b.OrderManager()

    def calculate_data(self):
        pass

    def next(self, tick: Dict):
        super().next(tick)
        self.calculate_data()
        if self.current_phase is None:
            self.current_phase = Phase(initiated_at=self.ticks.iloc[-1].name, strategy=self)
        else:
            self.current_phase.next()
        for phase in self.active_ph:
            phase.next()

    def _terminate_last_phase(self, reason):
        last_phase = self.current_phase
        if last_phase.status != Phase.STATUS_TERM:
            raise Exception("expected terminated status")
        if self.active_ph is not None and len(self.active_ph) > 0 and self.active_ph[-1].direction != last_phase.direction:
            self.current_phase = self.active_ph[-1]
            if self.current_phase.status == Phase.STATUS_RETRACING:
                self.current_phase.confirm()
            logger.info(f"Reactivated1 phase: {self.current_phase} at: {last_phase.ended_at}")
            del self.active_ph[-1]
        else:
            new_start = last_phase.get_new_phase_start()
            self.current_phase = Phase(initiated_at=new_start, strategy=self)
        self.inactive_ph.append(last_phase)

    def _retrace_last_phase(self, reason):
        last_phase = self.current_phase
        if last_phase.status != Phase.STATUS_RETRACING:
            raise Exception(f"expected {Phase.STATUS_RETRACING} status")
        if self.active_ph is not None and len(self.active_ph) > 0 and self.active_ph[-1].direction != last_phase.direction:
            self.current_phase = self.active_ph[-1]
            logger.info(f"Reactivated2 phase: {self.current_phase} at: {last_phase.ended_at}")
            if self.current_phase.status == Phase.STATUS_RETRACING:
                self.current_phase.confirm()
            del self.active_ph[-1]
        else:
            new_start = last_phase.get_new_phase_start()
            self.current_phase = Phase(initiated_at=new_start, strategy=self)
        self.active_ph.append(last_phase)

    def _stagnate_last_phase(self, reason):
        last_phase = self.current_phase
        if last_phase.status != Phase.STATUS_STAGNATING:
            raise Exception(f"expected {Phase.STATUS_STAGNATING} status")
        if self.active_ph is not None and len(self.active_ph) > 0 and self.active_ph[-1].direction != last_phase.direction:
            self.current_phase = self.active_ph[-1]
            # if self.current_phase.status == Phase.STATUS_RETRACING:
            #     self.current_phase.confirm()
            logger.info(f"Reactivated3 phase: {self.current_phase} at: {last_phase.ended_at}")
            del self.active_ph[-1]
        else:
            new_start = last_phase.get_new_phase_start()
            self.current_phase = Phase(initiated_at=new_start, strategy=self)
        self.active_ph.append(last_phase)
    
    def set_new_phase(self, reason):
        last_phase = self.current_phase
        if last_phase.status == Phase.STATUS_TERM:
            self._terminate_last_phase(reason)
        elif last_phase.status == Phase.STATUS_RETRACING:
            self._retrace_last_phase(reason)
        elif last_phase.status == Phase.STATUS_STAGNATING:
            self._stagnate_last_phase(reason)
        else:
            raise Exception("status not handled")

    def on_termination(self, phase, reason):
        logger.info(f"phase terminated: {phase}")
        if self.om.has_intrade_orders(phase):
            self.om.square_off_all_orders(index=self.ticks.iloc[-1].name, last_price=self.ticks.iloc[-1].last_price, phase=phase)
        if phase.id != self.current_phase.id:
            idx = [i for i, cphase in enumerate(self.active_ph) if cphase.id == phase.id][0]
            self.inactive_ph.append(self.active_ph[idx])
            del self.active_ph[idx]
            return
        self.set_new_phase(reason)
        # if self.current_phase.direction == UP and self.current_phase.status == Phase.STATUS_CONFIRMED:
        #     order = b.Order(type=b.Order.TYPE_BUY, limit_price=self.ticks.iloc[-1].last_price, created_at=self.ticks.iloc[-1].name, quantity=self.quantity, exchange_order_id=None)
        #     po = b.PhaseOrder(self.current_phase, order)
        #     self.om.place_order(po)
        #     logger.info(f"placed order for phase start: {self.current_phase}, order: {order}")

    def on_retracel(self, phase, reason):
        logger.info(f"phase retracing {phase}")
        if phase.id != self.current_phase.id:
            return
        if self.om.has_intrade_orders():
            self.om.square_off_all_orders(index=self.ticks.iloc[-1].name, last_price=self.ticks.iloc[-1].last_price, phase=self.current_phase)
        self.set_new_phase(reason)
        if self.current_phase.direction == UP:
            order = b.Order(type=b.Order.TYPE_BUY, limit_price=self.ticks.iloc[-1].last_price, created_at=self.ticks.iloc[-1].name, quantity=self.quantity, exchange_order_id=None)
            po = b.PhaseOrder(self.current_phase, order)
            self.om.place_order(po)
            logger.info(f"placed order for phase start: {self.current_phase}, order: {order}")

    def on_resume(self, phase, reason):
        logger.info(f"resumed the phase: {phase}")
        self.on_confirmed(phase, reason)

    def on_confirmed(self, phase, reason):
        logger.info(f"phase on confirmed {phase}")
        if phase.id != self.current_phase.id:
            return
        if self.current_phase.direction == UP and not (self.om.has_intrade_orders()):
            order = b.Order(type=b.Order.TYPE_BUY, limit_price=self.ticks.iloc[-1].last_price, created_at=self.ticks.iloc[-1].name, quantity=self.quantity, exchange_order_id=None)
            po = b.PhaseOrder(self.current_phase, order)
            self.om.place_order(po)
            logger.info(f"placed order for phase start: {self.current_phase}, order: {order}")

    def on_stagnation(self, phase, reason):
        logger.info(f"phase stagnating {phase}, slope: {phase.last_slope}")
        if phase.id != self.current_phase.id:
            return
        if self.om.has_intrade_orders():
            self.om.square_off_all_orders(index=self.ticks.iloc[-1].name, last_price=self.ticks.iloc[-1].last_price, phase=self.current_phase)
        # self.set_new_phase(reason)

ps = PhaseStrategy(instrument=instrument, settings=settings)
# for i in range(tdf.shape[0]):
# for i in range(1000, 3000):
for i in range(200):
    ps.next(tdf.iloc[i].to_dict())

print(ps.current_phase)
print(ps.inactive_ph)
print(ps.active_ph)

2024-03-21 18:07:33,959 - INFO - 2420517982.py:13 - [129859101840640] - 
created new phase0 at: 0
2024-03-21 18:07:33,966 - INFO - 2420517982.py:298 - [129859101840640] - 
phase on confirmed ==Phase 0:DOWN, CONFIRMED I:0, R: 0, C:None R:None, T:None, RJ:None, cr:0 tr: None, rr:None, end:1, vc: False, pc: False, st: None==
2024-03-21 18:07:33,972 - INFO - 2420517982.py:58 - [129859101840640] - 
is_significant: 0, at:2, ptick start: 0 mean: 3150.0, v: 1900.0, ch: 0.20000000000001705, d: DOWN
2024-03-21 18:07:33,974 - INFO - 2420517982.py:125 - [129859101840640] - 
is_retracing: 2, is_re: False, chv: 1900.0, vmax: 7550.0 vrf: 1
2024-03-21 18:07:33,978 - INFO - 2420517982.py:133 - [129859101840640] - 
is_stagnating: 2, is_st: False, chv: 1900.0, vmean: 3150.0 vsf: 2
2024-03-21 18:07:33,980 - INFO - 2420517982.py:298 - [129859101840640] - 
phase on confirmed ==Phase 0:DOWN, CONFIRMED I:0, R: 0, C:None R:None, T:None, RJ:None, cr:0 tr: None, rr:None, end:2, vc: False, pc: False, st: None==
2

Exception: this shouldn't be reached

In [3]:
print(f"Total orders: {len(ps.om.closed_orders)}")
print(f"Total Avg PC %: {sum([po.order.pnl_pc for po in ps.om.closed_orders]) / len(ps.om.closed_orders)}")

Total orders: 4
Total Avg PC %: 1.53557826558191


In [4]:
from bokeh.models import ColumnDataSource
all_phases = ps.inactive_ph + [phase for phase in ps.active_ph if phase.retraced_at is not None]
up_ini_idx = [ps.ticks.loc[phase.initiated_at].id for phase in all_phases if phase.direction == UP]
up_initiate = [ps.ticks.loc[phase.initiated_at].last_price for phase in all_phases if phase.direction == UP]

up_end_idx = [ps.ticks.loc[phase.retraced_at].id for phase in all_phases if phase.direction == UP]
up_end = [ps.ticks.loc[phase.retraced_at].last_price for phase in all_phases if phase.direction == UP]

down_ini_idx = [ps.ticks.loc[phase.initiated_at].id for phase in all_phases if phase.direction == DOWN]
down_initiate = [ps.ticks.loc[phase.initiated_at].last_price for phase in all_phases if phase.direction == DOWN]
down_end_idx = [ps.ticks.loc[phase.retraced_at].id for phase in all_phases if phase.direction == DOWN]
down_end = [ps.ticks.loc[phase.retraced_at].last_price for phase in all_phases if phase.direction == DOWN]

up_ini_plot = ColumnDataSource({'x': up_ini_idx, 'y': up_initiate})
up_end_plot = ColumnDataSource({'x': up_end_idx, 'y': up_end})

down_ini_plot = ColumnDataSource({'x': down_ini_idx, 'y': down_initiate})
down_end_plot = ColumnDataSource({'x': down_end_idx, 'y': down_end})

order_created_at = [ps.ticks.loc[po.order.created_at].id for po in ps.om.closed_orders]
order_created_at_price = [ps.ticks.loc[po.order.created_at].last_price for po in ps.om.closed_orders]

order_sqo_at = [ps.ticks.loc[po.order.square_off_at].id for po in ps.om.closed_orders]
order_sqo_at_price = [ps.ticks.loc[po.order.square_off_at].last_price for po in ps.om.closed_orders]
order_at_cds = ColumnDataSource({'x': order_created_at, 'y': order_created_at_price})
order_sqo_at_cds = ColumnDataSource({'x': order_sqo_at, 'y': order_sqo_at_price})

down_initiate = [ps.ticks.loc[phase.initiated_at].last_price for phase in all_phases if phase.direction == DOWN]

ticks_cdf = ColumnDataSource(ps.ticks)
# ut.bokeh_plot(cds=ticks_cdf, x_label="timestamp", y_label="price", plot='line', subplots=[up_ini_plot, up_end_plot, down_ini_plot, down_end_plot], subplot_labels=['Up Initiate', 'Up end', 'Down Initiate', 'Down end'])
# ut.bokeh_plot(cds=ticks_cdf, x_label="timestamp", y_label="price", plot='line', subplots=[up_ini_plot, up_end_plot, down_ini_plot, down_end_plot], subplot_labels=['Up Initiate', 'Up end', 'Down Initiate', 'Down end'])
ut.bokeh_plot(cds=ticks_cdf, x_label="timestamp", y_label="price", plot='line', subplots=[up_ini_plot, up_end_plot, down_ini_plot, down_end_plot, order_at_cds, order_sqo_at_cds], subplot_labels=['Up Initiate', 'Up end', 'Down Initiate', 'Down end', 'Order At', 'Order SQO'])

Loading BokehJS ...

In [17]:
ps.ticks["ch_vol"] = ps.ticks.volume
ps.ticks.loc[ps.ticks.ch < 0, "ch_vol"] = - ps.ticks.volume
ps.ticks["ch_vol"]

0            0.0
1        -7550.0
2         1900.0
3         2400.0
4        -6600.0
           ...  
196      -1500.0
197        400.0
198       2750.0
199       2350.0
ch_vol       NaN
Name: ch_vol, Length: 201, dtype: float64

In [24]:

tdf["ch"] = tdf.last_price.diff()
tdf["ch_vol"] = tdf.volume
tdf.loc[tdf.ch < 0, "ch_vol"] = - tdf.volume
# ps.ticks["vwch2"] = ps.ticks.ch * ps.ticks.volume
# ps.ticks.loc[ps.ticks.ch < 0, "vwch1"] = ps.ticks.loc[ps.ticks.ch < 0, "vwch1"] * -1
tdf["zero"] = 0
# ut.bokeh_series_plot(ps.ticks, 'vwch1', 'id')
# ut.bokeh_series_plot(ps.ticks, 'vwch2', 'id')
tdf.loc[(tdf.ch_vol > 10000) | (tdf.ch_vol < -10000)].plot.line(x='index', y=['ch_vol', 'zero'], width=2000, height=800, hover_cols=['volume', 'ch'])
# ps.ticks.plot.line(x='index', y='cum_vwch', width=2000, height=800)

:NdOverlay   [Variable]
   :Curve   [index]   (value,volume,ch)

In [18]:

ps.ticks["ch"] = ps.ticks.last_price.diff()
ps.ticks["ch_vol"] = ps.ticks.volume
ps.ticks.loc[ps.ticks.ch < 0, "ch_vol"] = - ps.ticks.volume
# ps.ticks["vwch2"] = ps.ticks.ch * ps.ticks.volume
# ps.ticks.loc[ps.ticks.ch < 0, "vwch1"] = ps.ticks.loc[ps.ticks.ch < 0, "vwch1"] * -1
ps.ticks["zero"] = 0
# ut.bokeh_series_plot(ps.ticks, 'vwch1', 'id')
# ut.bokeh_series_plot(ps.ticks, 'vwch2', 'id')
ps.ticks.loc[ps.ticks.ch_vol.notna()].plot.line(x='index', y=['ch_vol', 'zero'], width=2000, height=800, hover_cols=['volume', 'ch'])
# ps.ticks.plot.line(x='index', y='cum_vwch', width=2000, height=800)

:NdOverlay   [Variable]
   :Curve   [index]   (value,volume,ch)

In [4]:
# for i, row in ps.ticks.iterrows():

# ps.ticks['vwch_rolling'] = ps.ticks.vwch.rolling(window=5).sum()
# ps.ticks['vwch_cum'] = ps.ticks.vwch.cumsum()
ps.ticks.head(100)

,last_price,last_traded_quantity,total_buy_quantity,total_sell_quantity,last_trade_time,oi,volume,id
0,220.35,50,43000,45600,2024-03-15 09:15:02,1879800,0.0,0
1,218.10,250,56350,42950,2024-03-15 09:15:03,1879800,7550.0,1
2,218.30,200,73600,47500,2024-03-15 09:15:03,1879800,1900.0,1
3,223.15,50,89500,58800,2024-03-15 09:15:04,1879800,2400.0,2
4,218.10,50,95500,67350,2024-03-15 09:15:05,1879800,6600.0,3
...,...,...,...,...,...,...,...,...
76,224.95,50,168600,82900,2024-03-15 09:15:52,1906300,3800.0,50
77,224.90,100,157450,72000,2024-03-15 09:15:53,1906300,3500.0,51
78,224.40,50,166000,88700,2024-03-15 09:15:53,1906300,3100.0,51
79,226.80,300,174850,82250,2024-03-15 09:15:54,1906300,1400.0,52


# Per Quantity Movement Chart

In [20]:
print(ps.current_phase.pticks.vwch.iloc[:26].mean())
print(ps.current_phase.pticks.vwch.std())
# ps.current_phase.pticks.vwch

3162.799999999983
8518.726262539245


In [6]:
# from matplotlib import pyplot as plt

# plt.ion()
# plt.figure(figsize=(30, 12))  # Adjust the width and height as needed

# plt.plot(ps.ticks.loc[ps.ticks.pqch.notna()].pqch)

# ut.bokeh_series_plot(ps.ticks, 'vwch_cum', 'id')
# ps.inactive_ph[5]
# ps.inactive_ph[2].id
# ut.bokeh_series_plot(ps.inactive_ph[2].pticks, 'cum_vwch', 'id')
# ut.bokeh_series_plot(ps.current_phase.pticks, 'cum_vwch', 'id')
# ut.bokeh_series_plot(ps.current_phase.pticks, 'vwch', 'id')

ps.ticks["ch"] = ps.ticks.last_price.diff()
ps.ticks["vwch"] = ps.ticks.ch * ps.ticks.volume
ps.ticks["cum_vwch"] = ps.ticks.vwch.cumsum()
ut.bokeh_series_plot(ps.ticks, 'cum_vwch', 'id')

Loading BokehJS ...

In [14]:
ut.bokeh_series_plot(ps.current_phase.pticks, 'std_vwch', 'id')

Loading BokehJS ...

In [12]:
# print(ps.inactive_ph[5].pticks.to_json())
ps.inactive_ph[2].pticks['rolling_std'] = ps.inactive_ph[2].pticks.vwch.rolling(window=7).std()
print(ps.inactive_ph[2].pticks.vwch.abs().quantile(.7))
ps.inactive_ph[2].pticks

5125.000000000037


,last_price,last_traded_quantity,total_buy_quantity,total_sell_quantity,last_trade_time,oi,volume,id,ch,vwch,std_vwch,rolling_std
3,223.15,50,89500,58800,2024-03-15 09:15:04,1879800,2400.0,2,NaN,NaN,NaN,NaN
4,218.10,50,95500,67350,2024-03-15 09:15:05,1879800,6600.0,3,-5.05,-33330.0,NaN,NaN
5,222.70,50,104800,61750,2024-03-15 09:15:05,1879800,1850.0,3,4.60,8510.0,NaN,NaN
6,219.60,50,99900,76650,2024-03-15 09:15:06,1879800,6950.0,4,-3.10,-21545.0,NaN,NaN
7,218.30,100,108000,80000,2024-03-15 09:15:07,1879800,3350.0,5,-1.30,-4355.0,NaN,NaN
8,216.55,250,108800,82850,2024-03-15 09:15:08,1879800,1250.0,6,-1.75,-2187.5,16668.881899,NaN
9,215.30,50,106600,77100,2024-03-15 09:15:08,1879800,400.0,6,-1.25,-500.0,10954.304006,NaN
10,217.95,200,114050,67850,2024-03-15 09:15:09,1879800,3150.0,7,2.65,8347.5,10907.995262,15551.636063
11,216.30,50,106450,66550,2024-03-15 09:15:10,1879800,6750.0,8,-1.65,-11137.5,7042.553381,10642.914282
12,215.95,50,116000,73650,2024-03-15 09:15:10,1879800,600.0,8,-0.35,-210.0,6934.233691,9481.722549
